## セットアップ

In [ ]:
!pip install -q transformers sentencepiece sentence-transformers \
    torch faiss-cpu datasets scipy scikit-learn numpy

In [ ]:
# VRAMの環境変数設定
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

## モデルの準備

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("pfnet/plamo-embedding-1b", trust_remote_code=True)
model = AutoModel.from_pretrained("pfnet/plamo-embedding-1b", trust_remote_code=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


PlamoBiModel(
  (embed_tokens): Embedding(50112, 2048, padding_idx=3)
  (layers): ModifiedPlamoDecoder(
    (layers): ModuleList(
      (0-15): 16 x ModifiedPlamoDecoderLayer(
        (self_attn): ModifiedAttention(
          (qkv_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): DenseMLP(
          (gate_up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
        )
        (norm): RMSNorm()
        (norm2): RMSNorm()
      )
    )
  )
  (norm): RMSNorm()
)

## 埋め込み計算モジュールの定義

In [ ]:
import numpy as np

def embed_texts(
    texts: list[str],
    batch_size: int = 32,
    is_query: bool = False,
) -> np.ndarray:
    """
    texts           : 埋め込み対象の文字列リスト
    batch_size      : 一度に投入する文の数（デフォルト：32）
    is_query        : True の場合は model.encode_query を使用
                                False の場合は model.encode_document を使用
    """
    all_embs = []

    with torch.inference_mode():
      # 自動混合精度演算
      with torch.amp.autocast("cuda"):
            for i in range(0, len(texts), batch_size):
                batch = texts[i : i + batch_size]
                if is_query:
                    embs = model.encode_query(batch, tokenizer)
                else:
                    embs = model.encode_document(batch, tokenizer)

                # embs は torch.Tensor
                embs_np = embs.to("cpu").numpy()  # NumPy に変換
                # VRAM を開放する
                del embs
                torch.cuda.empty_cache()
                embs_np = np.nan_to_num(embs_np)
                all_embs.append(embs_np)

    return np.vstack(all_embs)


## JSTSによるモデルの評価

In [ ]:
from datasets import load_dataset

ds = load_dataset("sbintuitions/JMTEB", "jsts")
jsts = ds["test"]
print(jsts.column_names)  # ['sentence_pair_id', 'yjcaptions_id', 'sentence1', 'sentence2', 'label']

['sentence_pair_id', 'yjcaptions_id', 'sentence1', 'sentence2', 'label']


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# JSTS の全ペアを「文1／文2」と「スコア」に分解
sent1 = jsts["sentence1"]
sent2 = jsts["sentence2"]
gold_score = np.array(jsts["label"], dtype=float)  # 0〜5 の連続値

# 埋め込み
emb1 = embed_texts(sent1)
emb2 = embed_texts(sent2)

# コサイン類似度
# N×2048 の行列同士のコサイン類似度を一度に出す
sim_matrix = cosine_similarity(emb1, emb2)
# 対角成分だけを取り出せばペアごとの類似度が得られる
cos_sim = np.diag(sim_matrix)  # shape=(N,)

In [ ]:
# スピアマンとピアソンの相関係数を計算
from scipy.stats import spearmanr, pearsonr

spearman_corr, _ = spearmanr(cos_sim, gold_score)
pearson_corr, _  = pearsonr(cos_sim, gold_score)

print(f"Spearman: {spearman_corr:.4f}")
print(f"Pearson : {pearson_corr:.4f}")

Spearman: 0.6925
Pearson : 0.7379


## インデックス用データ読み込み・前処理

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
from datasets import load_dataset
# Livedoor ニュースコーパス (約7,300記事) を train/val/test に分割読み込み
ds = load_dataset(
    "shunk031/livedoor-news-corpus",
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    shuffle=False,
)
train = ds["train"]
print(train)

Dataset({
    features: ['url', 'date', 'title', 'content', 'category'],
    num_rows: 5894
})


In [ ]:
# 検索対象：記事のタイトル＋本文
corpus_texts = [
    f"{row['title']}。{row['content']}"
    for row in train
]

# （例）最初の5件をクエリとして流用
query_texts = corpus_texts[:5]
# 正解ID (今回は自身の記事をトップ1に返す recall@1)
true_ids = list(range(len(query_texts))) # true_ids = [0, 1, 2, ..., 5]

In [ ]:
# 記事のタイトルと本文(corpus_texts)をjsonl形式で保存

import json

with open('/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/corpus.jsonl', 'w', encoding='utf-8') as f:
    for text in corpus_texts:
        json.dump({'text': text}, f, ensure_ascii=False)
        f.write('\n')


In [ ]:
xb = embed_texts(corpus_texts, batch_size=8, is_query=False)  # index 用
xq = embed_texts(query_texts,  batch_size=8, is_query=True)   # query 用

In [ ]:
import faiss

d = xb.shape[1]  # 埋め込み次元
M = 32           # 各点の近傍リンク数
efC = 200        # 構築時の探索深さ

index = faiss.IndexHNSWFlat(d, M)
index.hnsw.efConstruction = efC

# 埋め込みベクトルを追加
index.add(xb)

In [ ]:
index.hnsw.efSearch = 50  # 検索時の探索深さ
k = 5                    # top-k 件取得
D, I = index.search(xq, k) # 検索を実行

In [ ]:
recall1 = np.mean([1 if true_ids[i] in I[i,:1] else 0 for i in range(len(true_ids))])
print(f"Recall@1: {recall1:.3f}")

Recall@1: 1.000


In [ ]:
# 埋め込みと Faiss インデックスを保存
np.save('/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/embeddings_xb.npy', xb)
faiss.write_index(index, '/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/hnsw_index.faiss')

In [ ]:
for i in range(len(query_texts)):
    print(f"Query: {query_texts[i]}")
    for rank, idx in enumerate(I[i]):
        print(f"Top {rank+1}: {corpus_texts[idx]}")
    print("========")

Query: 【韓国ニュース】「モテなかったから僧侶になった」　韓国僧侶の発言が話題呼ぶ。韓国では一般的に僧侶は独身であることが原則となっているが、「異性からモテず、絶望し出家を決意した」と語る僧侶のFacebookが韓国のネット掲示板で話題になっている。  ヒョボンという名前のこの僧侶は、19日、Facebookで「20代の私は異性に人気がありませんでした。（中略）結局どうしようもないという結論に至り、出家を決心しました。皆さんも諦めて出家しなさい。今になって考えれば若いころ、ラップやパンクをすべきだった」と、僧侶になった背景を語っている。  また、僧侶生活に関しては「早く出家するほど偉くなるのも早い。偉くなったらご飯と洗濯を悩まなくてもいい」とも語り、ほかにも「アイドルのPVはセクシーならばそれで十分。音楽性は問題ではない」「整形すること自体は問題ではない。整形しても可愛くないことが問題」など、僧侶らしからぬ投稿を続けている。  18日に開設されたばかりの彼のFacebookアカウントは、瞬く間に韓国ネットユーザーの間で話題となり、韓国ネット掲示板では「仏教徒に改宗しようかな」「本物のお坊さんではないのでは!?」「これは新たな宗教だね」など、様々な反響を呼んでいる。なお、ヒョボン氏が本物の僧侶か否かは未だ確認されていないようである。  【関連記事】 ・国仏教協会、韓国の曹渓宗組織委員会に憤慨の意 ・ユネスコ登録目指す韓国の『燃灯会』　街中が灯籠だらけ…ごり押しPRに自国内からも反発相次ぐ ・7月2週目韓国オンラインゲームランキング“韓国国民の妹”IU、日本のアイドルにライバル心？目指すは「隣の妹」
Top 1: 【韓国ニュース】「モテなかったから僧侶になった」　韓国僧侶の発言が話題呼ぶ。韓国では一般的に僧侶は独身であることが原則となっているが、「異性からモテず、絶望し出家を決意した」と語る僧侶のFacebookが韓国のネット掲示板で話題になっている。  ヒョボンという名前のこの僧侶は、19日、Facebookで「20代の私は異性に人気がありませんでした。（中略）結局どうしようもないという結論に至り、出家を決心しました。皆さんも諦めて出家しなさい。今になって考えれば若いころ、ラップやパンクをすべきだった」と、僧侶になった背景を語っている。  また、僧侶生活に関して

## 保存したインデックスを読み込んで使用

In [ ]:
!pip install faiss-cpu
import faiss
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
index = faiss.read_index('/content/drive/MyDrive/Colab Notebooks/data/text_embedding_search/hnsw_index.faiss')

In [ ]:
query_texts = ["渋谷の事件についてのニュース", "プロ野球の試合結果", "新作映画のレビュー"]
xq = embed_texts(query_texts, is_query=True)

In [ ]:
index.hnsw.efSearch = 50  # 検索時の探索深さ
k = 5  # 上位k件を取得
D, I = index.search(xq, k)  # Dは距離、Iはインデックス

In [ ]:
for i, q in enumerate(query_texts):
  print(f"\n[Query] {q}")
  for j in range(k):
    doc_id = I[i, j] # i番目のクエリにつき5つのID
    print(f" RANK {j+1}: {corpus_texts[doc_id]}")


[Query] 渋谷の事件についてのニュース
 RANK 1: 渋谷駅の駅ビルで通り魔。22日、16時過ぎ、東急東横線渋谷駅の駅ビル内で女性が刺されたとNHKニュースが報じた。女性に意識はあるという。  また、刺されたのは６０代の女性で救急車で病院に運ばれ、手当てを受けているという。女性を刺したと見られる７０代の女は、身柄を確保されたと同局が報じた。  Twitterでは、実際に事件現場を目撃した人たちのツイートも投稿されており「目の前に通り魔に刺されて血だらけの人がいる」「血尋常じゃないんだけど」と、騒然としている状態だ。  【関連情報】 ・【速報】渋谷で通り魔≪容疑者確保済≫（NAVERまとめ）
 RANK 2: まつゆう*お気に入りの隠れ家ガイド。スマホやPCから気軽にアクセスできる「ロケタッチガイド」は、地元の人や実際に旅行した人のおすすめスポット情報を集めたガイドブックです。他のガイドブックには載っていない、地域に密着した生の情報がいっぱい詰まっています。  この「ロケタッチガイド」を使って、まつゆう*さんにちょっと見つけにくいところにあるおすすめスポットガイドを作ってもらいました。  3万円分の旅行券が当たるキャンペーンも実施中なので、お見逃しなく！  まつゆう*の隠れ家 〜渋谷近辺 マッシューサロン 東京都渋谷区神南1-13-4 フレイムインボックス1F   渋谷の隠れ家的存在で、数々の有名人・業界人もこっそり通う知る人ぞ知る 「トラブル肌の駆け込み寺・マッシュー」  アトピー肌、トラブル肌にこそいい、オススメのフェイシャルトリートメントをしてくれたり、マッシュー オリジナルのスキンケアシリーズを購入できます。特にドラスティックゲル・EXは高いけど超おすすめの逸品！  オーナーのみどりさんとスタッフのヨセダさんは私にとって叔母的存在。 ここで心もお肌もリフレッシュ！ トリートメント関係なく、たまにお茶しに行ってます（笑）  ちなみにこの日はすっぴん♪ まつゆう*のお肌はマッシューのお陰で守られていると言っても過言じゃないです。  自分のブログでも1年使ってのレポート書いてます！これをみてたくさんの方が行って下さり「よくなったよ！」って感想を頂いてます。良かったら読んでみてください♡ 「トラブル肌の駆け込み寺"MASSHU(マッシュー)"・まとめページ

## gradioで検索機能を実装

In [ ]:
!pip install -q gradio

In [ ]:
import pandas as pd

# 検索用関数の定義
def search(query: str, k: int = 5, length: int = 30):
    # 1) クエリ埋め込み
    q_emb = embed_texts([query], is_query=True)
    # 2) Faiss 検索
    D, I = index.search(q_emb, k)
    # 3) 結果整形
    results = []
    for score, idx in zip(D[0], I[0]):
        title, snippet = corpus_texts[idx].split("。", 1)  # タイトル・本文を分割
        results.append({"title": title, "snippet": snippet[:length]+"…", "score": float(score)})

    df = pd.DataFrame(results) # gradio用にDFに変換

    return df


In [ ]:
search("パソコン", k=5)

,title,snippet,score
0,あなたは大丈夫？ 40代会社員はパソコンが使えない人多し！【話題】,会社で仕事をしていると、自分より年上で経験も多い40代の社員…,18906.625000
1,パソコンはもういらない？ スマホの普及は３倍もパソコンは減少【話題】,総務省の通信利用動向調査によると２０１１年末のスマートフォン…,20534.449219
2,使いこなし指南の知っ得！虎の巻【Wordテクニック集】,「パソコンの知識がある」という言葉には二種類の意味があると筆…,21659.353516
3,字が汚い人には朗報か？ メモ入りの便利な付箋が作れる！ カシオが「memopri（メモプリ）...,付箋紙にちょっとしたメモを残す人は多いだろう。カシオが発表し…,21711.826172
4,アップルに対抗できるのか? マイクロソフトの独自タブレット【デジ通】,Windowsで圧倒的なシェアを誇っていたマイクロソフトは、…,21784.621094


In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=search,
    inputs=[
        gr.Textbox(lines=2, placeholder="検索クエリを入力"),
        gr.Slider(minimum=1, maximum=10, step=1, label="Top k 件数")
    ],
    outputs=gr.Dataframe(
        headers=["title", "snippet", "score"],
        row_count=5
    ),
    title="Livedoor ニュース検索デモ",
    description="pfnet/plamo-embedding-1b + Faiss HNSW による類似ニュース検索"
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3581c5774baec4dc39.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
